In [ ]:
#imports (reminder to come back and clean this up)
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import scipy.io
import numpy as np
import glob
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Sequential
import os
from PIL import Image
import gc

In [ ]:
#load data
data = scipy.io.loadmat('G:\\\\capstone files\\\\data\\\\archive (1)\\\\cars_annos.mat')

In [ ]:
data.keys()

the only two I care about is "annotations" and "class_names"

In [ ]:
class_names = pd.DataFrame(data['class_names'])
class_names.head()

In [ ]:
data['annotations']

In [ ]:
#make dataframes for each column to make it easier to work with
relative_im_path = pd.DataFrame(data['annotations']['relative_im_path']).transpose()
bbox_x1 = pd.DataFrame(data['annotations']['bbox_x1']).transpose()
bbox_x2 = pd.DataFrame(data['annotations']['bbox_x2']).transpose()
bbox_y1 = pd.DataFrame(data['annotations']['bbox_y1']).transpose()
bbox_y2 = pd.DataFrame(data['annotations']['bbox_y2']).transpose()
class_id = pd.DataFrame(data['annotations']['class']).transpose()
test = pd.DataFrame(data['annotations']['test']).transpose()


In [ ]:
#concatenate the dataframes into one
annotations = pd.concat([relative_im_path, bbox_x1, bbox_y1, bbox_x2, bbox_y2, class_id, test], axis=1)
annotations.columns = ['relative_im_path', 'x1', 'y1', 'x2', 'y2', 'class_id', 'test']
annotations.head()
#values are in the same order, confirmed through the matlab file using octave

In [ ]:
annotations.info()

In [ ]:
#cast all columns to int except for relative_im_path
annotations[annotations.columns.difference(['relative_im_path'])] = annotations[annotations.columns.difference(['relative_im_path'])].astype(int)

In [ ]:
annotations

In [ ]:
import glob
import os
from PIL import Image
#set the paths for the images
train_image_paths = glob.glob('G:/capstone files/data/archive (1)/cars_train/cars_train/*.jpg')
test_image_paths = glob.glob('G:/capstone files/data/archive (1)/cars_test/cars_test/*.jpg')

#resize all the images to 224x224 and store in local memory to save ram
def save_resized_images(image_paths, output_dir, size=(224, 224)):
    for image_paths in image_paths:
        with Image.open(image_paths) as img:
            resized_img = img.resize(size)
            output_path = os.path.join(output_dir, os.path.basename(image_paths))
            resized_img.save(output_path)
#set the output directories for the resized images
train_output_dir = "G:/capstone files/data/cars train resized"
test_output_dir = "G:/capstone files/data/cars test resized"
#make the directories if they don't exist
if not os.path.exists(train_output_dir):
    os.makedirs(train_output_dir)
if not os.path.exists(test_output_dir):
    os.makedirs(test_output_dir)

save_resized_images(train_image_paths, train_output_dir)
save_resized_images(test_image_paths, test_output_dir)

In [ ]:
#get new paths for the resized images
resized_train_image_path = glob.glob(train_output_dir + '/*.jpg')
resized_test_image_path = glob.glob(test_output_dir + '/*.jpg')

In [ ]:

#preprocess the images and store them in a dataset using tensorflow data api
def preprocess_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image)
    image = tf.image.convert_image_dtype(image, tf.float32)
    return image

train_images = tf.data.Dataset.from_tensor_slices(resized_train_image_path)
train_images = train_images.map(preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
test_images = tf.data.Dataset.from_tensor_slices(resized_test_image_path)
test_images = test_images.map(preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [ ]:
# Load the first image in the training set
first_image = list(train_images.take(1))[0]
#check to see if it is the right size
first_image_shape = first_image.shape
print(first_image_shape)


In [ ]:

train_annotations = annotations.drop(annotations[annotations['test'] == 1].index)
test_annotations = annotations.drop(annotations[annotations['test'] == 0].index)
